In [60]:
import os 
import json
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso

from sklearn.pipeline import make_pipeline

import warnings
#warnings.filterwarnings("ignore")

In [11]:
!ls /mnt/C89CF3A00D398BCA/Users/HOME/Programming/Entrance\ tasks/Cinimex\ Data\ Lab/data

student-mat.csv  student-merge.R  student-por.csv  student.txt


In [52]:
df = pd.read_csv("../data/student-mat.csv", ";")
print(df.shape)
df.head(1)

(395, 33)


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6


In [53]:
numeric = ["age", 'Medu', "Fedu", "traveltime", "studytime", "failures",\
           "famrel", "freetime", "goout", "Dalc", "Walc", "health", "absences", "G1", "G2"]

X, y = df[numeric], df["G3"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)

In [55]:
df.loc[df["age"] >= 20, 'age'] = 19

In [56]:
df[df["age"] >= 20]

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3


In [66]:
def build_and_train():
    # loading data
    df = pd.read_csv("../data/student-mat.csv", ";")
    numeric_cols = ["age", 'Medu', "Fedu", "traveltime", "studytime", "failures",
                    "famrel", "freetime", "goout", "Dalc", "Walc", "health", "absences", "G1", "G2"]

    x, y = df[numeric_cols], df["G3"]
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.1, random_state=42)

    # making pipeline
    pipe = make_pipeline(PreProcessing(), Lasso(alpha=0.0001))
                         #RandomForestRegressor())

    # setting up grid search
    param_grid = {"randomforestregressor__n_estimators": [10, 20, 30],
                  "randomforestregressor__max_depth": [None, 6, 8, 10],
                  "randomforestregressor__max_leaf_nodes": [None, 5, 10, 20],
                  "randomforestregressor__min_impurity_split": [0.1, 0.2, 0.3]}
    
    param_grid = {}

    # running grid search
    grid = GridSearchCV(pipe, param_grid=param_grid, cv=3)
    grid.fit(x_train, y_train)

    return grid

class PreProcessing(BaseEstimator, TransformerMixin):
    """Custom Pre-Processing estimator for our use-case
    """

    def __init__(self):
        pass

    def transform(self, df):
        """Regular transform() that is a help for training, validation & testing datasets
           (NOTE: The operations performed here are the ones that we did prior to this cell)
        """
        numeric_cols = ["age", 'Medu', "Fedu", "traveltime", "studytime", "failures",
                        "famrel", "freetime", "goout", "Dalc", "Walc", "health", "absences", "G1", "G2"]
        df = df[numeric_cols]
        
        df.loc[df["age"] >= 20, 'age'] = 19
        df.loc[df["Dalc"] > 3, 'Dalc'] = 3
        
        return df.as_matrix()

    def fit(self, df, y=None, **fit_params):
        """Fitting the Training dataset & calculating the required values from train
           e.g: We will need the mean of X_train['Loan_Amount_Term'] that will be used in
                transformation of X_test
        """
        return self
    
grid = build_and_train()

from sklearn import metrics
df = pd.read_csv("../data/student-por.csv", ";")
numeric_cols = ["age", 'Medu', "Fedu", "traveltime", "studytime", "failures",
                        "famrel", "freetime", "goout", "Dalc", "Walc", "health", "absences", "G1", "G2"]
X, y = df[numeric_cols], df["G3"]
metrics.mean_absolute_error(y_test, grid.predict(X_test))

1.494434216789609

In [39]:
df.drop(df[df["G3"] == 0].index)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
5,GP,M,16,U,LE3,T,4,3,services,other,...,5,4,2,1,2,5,10,15,15,15
6,GP,M,16,U,LE3,T,2,2,other,other,...,4,4,4,1,1,3,0,12,12,11
7,GP,F,17,U,GT3,A,4,4,other,teacher,...,4,1,4,1,1,1,6,6,5,6
8,GP,M,15,U,LE3,A,3,2,services,other,...,4,2,2,1,1,1,0,16,18,19
9,GP,M,15,U,GT3,T,3,4,other,other,...,5,5,1,1,1,5,0,14,15,15


In [33]:
#making pipeline
pipe = make_pipeline(PreProcessing(), RandomForestRegressor())
pipe

Pipeline(steps=[('preprocessing', PreProcessing()), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))])

In [16]:
#setting up grid search
param_grid = {"randomforestregressor__n_estimators" : [10, 20, 30],
             "randomforestregressor__max_depth" : [None, 6, 8, 10],
             "randomforestregressor__max_leaf_nodes": [None, 5, 10, 20], 
             "randomforestregressor__min_impurity_split": [0.1, 0.2, 0.3]}

#running grid search
grid = GridSearchCV(pipe, param_grid=param_grid, cv=3)

grid.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(steps=[('preprocessing', PreProcessing()), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'randomforestregressor__n_estimators': [10, 20, 30], 'randomforestregressor__max_depth': [None, 6, 8, 10], 'randomforestregressor__max_leaf_nodes': [None, 5, 10, 20], 'randomforestregressor__min_impurity_split': [0.1, 0.2, 0.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [34]:
print("Best parameters: {}".format(grid.best_params_))
print()
print("Validation set score: {:.2f}".format(grid.score(X_test, y_test)))

Best parameters: {'randomforestregressor__max_depth': 8, 'randomforestregressor__max_leaf_nodes': 20, 'randomforestregressor__min_impurity_split': 0.3, 'randomforestregressor__n_estimators': 30}

Validation set score: 0.82


In [40]:
#saving model
import dill as pickle
filename = 'model_v1.pk'

with open('flask_api/models/' + filename, 'wb') as file:
    pickle.dump(grid, file)

test_df = X_test
with open("flask_api/models/model_v1.pk", "rb") as file:
    loaded_mdoel = pickle.load(file)
loaded_mdoel.predict(test_df)

array([  7.65771791,  11.75536448,   6.8279828 ,   9.21527288,
         9.21960747,  12.74024116,  18.43117025,   7.36828981,
         7.50369813,  13.68939447,  15.53908385,   7.27364417,
        14.13919495,  11.57698879,  14.13947566,   8.10173326,
         1.35914299,  10.78393929,  15.2749301 ,   4.39619957,
        13.89660097,  15.3947856 ,  15.1313949 ,   6.5337037 ,
         6.92644074,  18.96553773,  10.9002691 ,  10.03307091,
        17.87701681,  10.92557405,   9.07976301,   8.66348207,
        15.1618546 ,  12.97659658,   1.651237  ,   6.54941138,
         0.        ,  15.30237821,  11.78838443,   7.92424414])

In [45]:
# testing server
import json
import requests

"""Setting the headers to send and accept json responses
"""
header = {'Content-Type': 'application/json', \
                  'Accept': 'application/json'}

"""Reading test batch
"""
#df = pd.read_csv('../data/test.csv', encoding="utf-8-sig")
#df = df.head()

"""Converting Pandas Dataframe to json
"""
data = test_df.to_json(orient='records')

In [52]:
"""POST <url>/predict
"""
#link = """https://deploying-predictions-tutorial.herokuapp.com/predict"""
link = """http://0.0.0.0:8007/predict"""

resp = requests.post(link, \
                    data = json.dumps(data),\
                    headers= header)

In [53]:
resp.status_code

200

In [54]:
"""The final response we get is as follows:
"""
resp.json()

{'predictions': '[{"0":5.7765337903},{"0":5.6044252044},{"0":5.7998220298},{"0":4.9475787638},{"0":5.6563564214},{"0":5.6044252044},{"0":5.9551112314},{"0":5.9960575998},{"0":1.4},{"0":4.8649657287},{"0":5.8620556758},{"0":5.9923538961},{"0":5.5664622415},{"0":5.4775733526},{"0":5.7218326118},{"0":1.8},{"0":5.9024597162},{"0":5.8320893458},{"0":4.5554834055},{"0":5.8654226792},{"0":4.5892328042},{"0":4.4804834055},{"0":5.8695893458},{"0":6.0783387446},{"0":5.780700457},{"0":5.9923538961},{"0":5.6044252044},{"0":4.731758658},{"0":5.9139075277},{"0":1.2},{"0":6.0783387446},{"0":5.4822029822},{"0":5.5155363155},{"0":5.8168783069},{"0":5.6044252044},{"0":5.6563564214},{"0":5.817737494},{"0":6.0725186388},{"0":5.5525733526},{"0":5.7588696489}]'}

In [58]:
resp.json()["predictions"]

'[{"0":5.7765337903},{"0":5.6044252044},{"0":5.7998220298},{"0":4.9475787638},{"0":5.6563564214},{"0":5.6044252044},{"0":5.9551112314},{"0":5.9960575998},{"0":1.4},{"0":4.8649657287},{"0":5.8620556758},{"0":5.9923538961},{"0":5.5664622415},{"0":5.4775733526},{"0":5.7218326118},{"0":1.8},{"0":5.9024597162},{"0":5.8320893458},{"0":4.5554834055},{"0":5.8654226792},{"0":4.5892328042},{"0":4.4804834055},{"0":5.8695893458},{"0":6.0783387446},{"0":5.780700457},{"0":5.9923538961},{"0":5.6044252044},{"0":4.731758658},{"0":5.9139075277},{"0":1.2},{"0":6.0783387446},{"0":5.4822029822},{"0":5.5155363155},{"0":5.8168783069},{"0":5.6044252044},{"0":5.6563564214},{"0":5.817737494},{"0":6.0725186388},{"0":5.5525733526},{"0":5.7588696489}]'

In [59]:
pd.read_json(resp.json())

ValueError: Invalid file path or buffer object type: <class 'dict'>

In [63]:
from pandas.io.json import json_normalize
json_normalize(resp.json(), sep=",")

,predictions
0,"[{""0"":5.7765337903},{""0"":5.6044252044},{""0"":5...."


In [51]:
t = loaded_mdoel.predict(test_df)
pd.DataFrame(t)

,0
0,7.657718
1,11.755364
2,6.827983
3,9.215273
4,9.219607
5,12.740241
6,18.431170
7,7.368290
8,7.503698
9,13.689394
